In [16]:
import subprocess
import imageio
import numpy as np
import matplotlib.pyplot as plt
import os
import time

In [17]:
def read_grids_from_file(grid_file):
    with open(grid_file, 'r') as file: #opens grid.txt
        lines = file.readlines()

    grids = []
    current_grid = []
    for line in lines: #reads the grid from the file
        if line.strip() == "":
            if current_grid:
                grids.append(np.array(current_grid))
                current_grid = []
        else:
            current_grid.append(list(map(int, line.strip().split())))

    if current_grid:
        grids.append(np.array(current_grid))

    return grids

In [50]:
def visualize_grid(grid, cost, output_file):
    fig, ax = plt.subplots(figsize=(8, 6))  # Create a figure and an axis object
    cmap = plt.cm.viridis
    cmap.set_bad(color='black')
    masked_grid = np.ma.masked_where(grid == -1, grid)
    
    cax = ax.matshow(masked_grid, cmap=cmap, interpolation='nearest')
    
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            if grid[i, j] != -1:
                ax.text(j, i, str(grid[i, j]), va='center', ha='center', color='white', fontsize=4)
            else:
                ax.add_patch(plt.Rectangle((j - 0.5, i - 0.5), 1, 1, fill=True, color='black'))

    # Display the cost on the figure
    fig.text(0.5, 0.05, f'Total Cost: {cost}', va='top', ha='center', fontsize=12, color='black')

    plt.savefig(output_file)
    plt.close()


In [19]:
def read_cost_from_file(cost_file):
    with open(cost_file, 'r') as file:
        return file.read().strip()


In [20]:
def create_gif_from_grids(grids, gif_file, cost_file, temp_dir='temp_frames', duration=0.5):
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)

    cost = read_cost_from_file(cost_file)  # Read the total cost
    frame_files = []
    for i, grid in enumerate(grids):
        frame_file = os.path.join(temp_dir, f'frame_{i:04d}.png')
        visualize_grid(grid, cost, frame_file)  # Pass the cost to the visualize function
        frame_files.append(frame_file)

    with imageio.get_writer(gif_file, mode='I', duration=duration) as writer:
        for frame_file in frame_files:
            image = imageio.imread(frame_file)
            writer.append_data(image)

    for frame_file in frame_files:
        os.remove(frame_file)
    os.rmdir(temp_dir)


In [21]:
def clear_file(file_path): #clear grids.txt for following run
    with open(file_path, 'w') as file:
        pass  # Opening in 'w' mode will truncate the file

In [22]:
def run_command(command):
    try:
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("Command output:", result.stdout.decode())
        print("Command error (if any):", result.stderr.decode())
    except subprocess.CalledProcessError as e:
        print("Error executing command:", e)
        print("Command output:", e.stdout.decode())
        print("Command error:", e.stderr.decode())

In [40]:
command = "g++ -O3 -o main main.cpp"
run_command(command)

Command output: 
Command error (if any): main.cpp:85:8: warning: 'auto' type specifier is a C++11 extension [-Wc++11-extensions]
  for (auto cell : nets[net]) {
       ^
main.cpp:85:18: warning: range-based for loop is a C++11 extension [-Wc++11-extensions]
  for (auto cell : nets[net]) {
                 ^
main.cpp:133:16: warning: 'auto' type specifier is a C++11 extension [-Wc++11-extensions]
    for (const auto& row : grid) {
               ^
main.cpp:133:26: warning: range-based for loop is a C++11 extension [-Wc++11-extensions]
    for (const auto& row : grid) {
                         ^
main.cpp:134:20: warning: 'auto' type specifier is a C++11 extension [-Wc++11-extensions]
        for (const auto& cell : row) {
                   ^
main.cpp:134:31: warning: range-based for loop is a C++11 extension [-Wc++11-extensions]
        for (const auto& cell : row) {
                              ^
main.cpp:196:14: warning: 'auto' type specifier is a C++11 extension [-Wc++11-extensions

In [51]:
run_command("./main")
time.sleep(2)
grid_file = 'grid.txt'  # Path to the grid updates text file
gif_file = 'grid_evolution.gif'  # Output GIF file name
cost_file = 'cost.txt'

grids = read_grids_from_file(grid_file)
create_gif_from_grids(grids, gif_file, cost_file)
clear_file(grid_file)
os.remove(cost_file)

Command output: Final Grid: 
-- -- -- -- -- -- -- 1126 -- 840 1259 372 810 118 806 114 1130 214 -- 96 885 921 198 409 392 -- 401 247 316 1076 -- -- 1101 997 516 493 100 94 01 624 622 -- -- 117 -- 97 708 115 -- -- 
-- -- -- -- 686 1088 156 1127 397 470 231 728 714 838 769 851 248 945 203 1032 914 518 242 26 519 479 843 123 440 162 696 -- 1265 271 606 618 341 629 698 17 1050 621 845 -- 142 00 151 -- -- -- 
-- -- -- 687 413 684 566 78 1271 472 471 758 464 506 707 771 501 1256 379 258 1037 548 168 544 991 568 1269 878 1140 617 616 642 656 735 620 737 513 320 308 1270 962 307 802 941 -- 102 -- -- 111 -- 
-- 863 30 839 813 1161 483 1134 09 823 1052 1171 796 601 505 1248 31 450 850 486 336 727 1151 976 611 785 951 368 763 414 439 922 197 191 294 628 216 1042 309 387 495 48 485 436 -- -- 229 721 -- -- 
-- 815 816 814 317 692 908 310 154 825 948 824 1197 798 133 749 391 1082 572 571 1157 299 355 444 916 880 777 327 1258 150 832 1288 626 473 712 634 511 857 1159 697 887 302 279 780 52 801 -- 98 

/var/folders/rj/rk3nbk3537g9r7jt8g9_3chw0000gn/T/ipykernel_1661/1836250826.py:14: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(frame_file)
